# Question 1

## (a) After solving the linear program, what is the optimal transportation cost?

In [210]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [211]:
model = gb.Model("Can2Oil")

In [212]:
Capacity_for_Direct_Production_Facilities = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Capacity_for_Direct_Production_Facilities.csv")
Capacity_for_Transship_Distribution_Centers = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Capacity_for_Transship_Distribution_Centers.csv")
Capacity_for_Transship_Production_Facilities = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Capacity_for_Transship_Production_Facilities.csv")

Cost_Production_to_Refinement = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Cost_Production_to_Refinement.csv")
Cost_Production_to_Transshipment = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Cost_Production_to_Transshipment.csv")
Cost_Transshipment_to_Refinement = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Cost_Transshipment_to_Refinement.csv")

Refinement_Demand = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 1 Files\Refinement_Demand.csv")

In [213]:
# Extract the "Cost" column 
direct_shipping_costs = Cost_Production_to_Refinement['Cost'].values.reshape(25, -1)
shipping_to_transsipment_costs = Cost_Production_to_Transshipment['Cost'].values.reshape(15, -1)
transsipment_to_refinement_costs = Cost_Transshipment_to_Refinement['Cost'].values.reshape(2, -1)

# Extract the "Capacity" and "Demand" column
direct_shipping_supply_capacity = Capacity_for_Direct_Production_Facilities['Capacity'].values.reshape(25, -1)
transshipment_supply_capacity = Capacity_for_Transship_Distribution_Centers['Capacity'].values.reshape(2, -1)
shipping_to_transshipment_supply_capacity = Capacity_for_Transship_Production_Facilities['Capacity'].values.reshape(15, -1)

refinement_demand = Refinement_Demand['Demand'].values.reshape(5, -1)

In [214]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [215]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [216]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

C:\Users\johns\AppData\Local\Temp\ipykernel_22968\29224195.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")


In [217]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

C:\Users\johns\AppData\Local\Temp\ipykernel_22968\1905600687.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")


In [218]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

C:\Users\johns\AppData\Local\Temp\ipykernel_22968\2124321848.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
C:\Users\johns\AppData\Local\Temp\ipykernel_22968\2124321848.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [219]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [220]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

C:\Users\johns\AppData\Local\Temp\ipykernel_22968\4255670903.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


In [221]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x7f8fadb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [222]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", model.status)

# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Model Status:  2
Number of Decision Variables:  165


### Answer to A: 24188.99

In [223]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  24188.59


## (b) In the optimal solution, what proportion of canola oil is transshipped?

In [224]:
# Get the optimal values of decision variables
to_transshipment_values = model.getAttr('x', y)

values_array = np.array(list(to_transshipment_values.values()))

In [225]:
# Calculate the mean
mean_value = np.mean(values_array)
mean_value

92.33333333333333

In [226]:
# Get the optimal values of decision variables
transshipment_values = model.getAttr('x', z)

In [227]:
# Calculate the total amount of canola oil transshipped
total_transshipped = sum(transshipment_values[i, j] for i in range(2) for j in range(5))

In [228]:
# Get the optimal values of decision variables
direct_shipping_values = model.getAttr('x', x)

In [229]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

5958.0

In [230]:
# Calculate the total amount of canola oil directly shipped for North America
direct_shipping_for_f = sum(direct_shipping_values[i, j] for i in range(15) for j in range(5))
direct_shipping_for_f

3706.0

### Answer to B: 0.31736938588450964

In [231]:
# Calculate the proportion
total_canola_oil = direct_shipping + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil
proportion_transshipped

0.31736938588450964

# Question 2

In [232]:
from gurobipy import GRB
import gurobipy as gb

In [233]:
model = gb.Model("SunnyShore Bay")

In [234]:
C = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Cash Balance")
B = model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")

In [235]:
model.setObjective(1.0175 * B[0] + 1.0225 * B[1] + 1.0275 * B[2] + 1.0175 * B[3] + 1.0225 * B[4] + 1.0175 * B[5], GRB.MINIMIZE)

### (a) How many different investments can be made over the 4-month period? 

## 6

In [236]:
model.addConstr(C[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May")
model.addConstr(C[1] == C[0] + 260000 - 400000 + B[3] + B[4] - 1.0175 * B[0], "June")
model.addConstr(C[2] == C[1] + 420000 - 350000 + B[5] - 1.0225 * B[1] - 1.0175 * B[3], "July")
model.addConstr(C[3] == C[2] + 580000 - 200000 - 1.0275 * B[2] - 1.0225 * B[4] - 1.0175 * B[5], "August")

<gurobi.Constr *Awaiting Model Update*>

In [237]:
model.addConstr(B[0] + B[1] + B[2] <= 250000, "Borrow_Limit_May")
model.addConstr(B[3] + B[4] <= 150000, "Borrow_Limit_June")
model.addConstr(B[5] <= 350000, "Borrow_Limit_July")

<gurobi.Constr *Awaiting Model Update*>

In [238]:
model.addConstr(C[0] >= 25000, "Min_Cash_Balance_May")
model.addConstr(C[1] >= 20000, "Min_Cash_Balance_June")
model.addConstr(C[2] >= 35000, "Min_Cash_Balance_July")
model.addConstr(C[3] >= 18000, "Min_Cash_Balance_August")

<gurobi.Constr *Awaiting Model Update*>

In [239]:
model.addConstr(C[2] >= 0.65 * (C[0] + C[1]), "July_Ratio_Constraint")

<gurobi.Constr *Awaiting Model Update*>

### (b) Write down the cash balance constraint for money on-hand at the end of June.

## z2 = z1 + b23 + b24 − 1.0175b12 − 140


### (c) Write down the linear ratio constraint associated with the cash balance at the end of July.

## z3 ≥ 0.65(z1 + z2)

In [240]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x42c1a00f
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 1 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.429047297e+05


In [241]:
print("Model Status: ", model.status)
print("Number of Decision Variables: ", model.numVars)

Model Status:  2
Number of Decision Variables:  10


### (d) What is the total amount that Sunnyshore Bay has to repay to the bank over the entire season?


## 142,904.73

In [242]:
print("Total Amount of Money: ", round(model.objVal, 2)) 

Total Amount of Money:  142904.73


### (e) How much money does Sunnyshore Bay withdraw in May from all loans?

## 5000

### (f) What is the cash balance at the end of August?

## 327,095

In [243]:
print(model.printAttr('X'))


    Variable            X 
-------------------------
Cash Balance[0]        25000 
Cash Balance[1]        20000 
Cash Balance[2]        35000 
Cash Balance[3]       327095 
   Borrow[2]         5000 
   Borrow[3]      54054.1 
   Borrow[4]      80945.9 
None


In [244]:
June_model = gb.Model("SunnyShore Bay June Increase")

C = June_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Cash Balance")
B = June_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")

June_model.setObjective(1.0175 * B[0] + 1.0225 * B[1] + 1.0275 * B[2] + 1.0175 * B[3] + 1.0225 * B[4] + 1.0175 * B[5], GRB.MINIMIZE)

June_model.addConstr(C[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May")
June_model.addConstr(C[1] == C[0] + 260000 - 400000 + B[3] + B[4] - 1.0175 * B[0], "June")
June_model.addConstr(C[2] == C[1] + 420000 - 350000 + B[5] - 1.0225 * B[1] - 1.0175 * B[3], "July")
June_model.addConstr(C[3] == C[2] + 580000 - 200000 - 1.0275 * B[2] - 1.0225 * B[4] - 1.0175 * B[5], "August")

June_model.addConstr(B[0] + B[1] + B[2] <= 250000, "Borrow_Limit_May")
June_model.addConstr(B[3] + B[4] <= 150000, "Borrow_Limit_June")
June_model.addConstr(B[5] <= 350000, "Borrow_Limit_July")

June_model.addConstr(C[0] >= 25000, "Min_Cash_Balance_May")
June_model.addConstr(C[1] >= 27500, "Min_Cash_Balance_June")
June_model.addConstr(C[2] >= 35000, "Min_Cash_Balance_July")
June_model.addConstr(C[3] >= 18000, "Min_Cash_Balance_August")

June_model.addConstr(C[2] >= 0.65 * (C[0] + C[1]), "July_Ratio_Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [245]:
June_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x20464e7e
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 1 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.906250e+04   0.000000e+00      0s
       6    1.5053662e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.505366247e+05


### (g) Due to potential unexpected repairs, one of the managers has suggested increasing the minimum cash balance for June to $27,500. How much will now have to be repaid if this change is approved?

## 150,536.62

In [246]:
print("Model Status: ", June_model.status)
print("Number of Decision Variables: ", June_model.numVars)

print("Total Amount of Money: ", round(June_model.objVal, 2)) 
print(June_model.printAttr('X'))

Model Status:  2
Number of Decision Variables:  10
Total Amount of Money:  150536.62

    Variable            X 
-------------------------
Cash Balance[0]        25000 
Cash Balance[1]        27500 
Cash Balance[2]        35000 
Cash Balance[3]       326963 
   Borrow[2]         5000 
   Borrow[3]      61425.1 
   Borrow[4]      81074.9 
None
